# Data Enrichment

Data enrichment and data aggregation are the processes involved around joining merging datasets, creating new columns, calculating values on certain windows, grouping into bins, or even changing the values. All of these processes will assist in the data analysis by providing specific insight into the data. Things such as severity of anomalous data, rolling averages, cumulative sums, or quantities grouped by ages can all make the data easier to interpret. We'll focus on the following for this class:
- Querying and Merging dataframes
- Aggregating Dataframes


In [ ]:
import pandas as pd

In [ ]:
weather = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/nyc_weather_2018.csv')

In [ ]:
snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head()

As a reminder, in previous chapters we have used the following syntax to generate the same output. 

In [ ]:
snow_data_mask = weather[
    (weather.datatype == 'SNOW') & (weather.value > 0)
]
snow_data_mask.head()

In [ ]:
snow_data_mask.equals(snow_data)

The method that you choose largely depends on preference. 

## Merging DataFrames

Another crutial part of data analysis is combining datasets together, creating a more complete understanding of the data. There are two types of merges that we typically talk about. Using the venacular common in from SQL: 
- `join`: a join is where you two sets of columns through one or values in the respective rows matching on some condition. 
- `union`: a merge where the columns are the same on two datasets, and bundled together on the same table.

Join is the more complex between the two, so we'll mostly talk about that.

### Joining Datasets
So far in this class, we have only worked with a single dataset. Joins provide us the ability to take two seperate tables or dataframes with related information, and combine them into a single table. For the weather data we've been using, we might perform a join to attach a physical location to the weather measurements using the weather station's id to gain a bettr idea of how the weather patterns are destributed.

The most common types of joins are described below using venn diagrams.  

![joins diagram](Assets/joins.jpg "Joins Diagram")

Think of the circles as the complete set of rows for each dataframe and the shaded region as the rows that are returned as a result of the join. Described breifly, 
- Inner joins return only the rows that are present in both dataframes, 
- Left (Right) joins return all of the rows from the left (right) dataframe, leaving all the values as null or missing (depending on your language) for the columns from the right (left) table if there was no matching ID found in that table
- Full or Outer joins return all rows from both tables, leaving missing values in the columns on both sides for missing IDs in either table (Think the left and right joins happening at the same time)

Since it can be easier to understand something by doing, let's at least look at some examples:

In [ ]:
# reading in the weather station data
stations = pd.read_csv("https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/weather_stations.csv")

In [ ]:
stations.head()

The first thing we need to do to join two dataframes is determine the rows that we can join on. Note the row called ID. This looks nearly identical to the type of values we had in some of the weather data. 

In [ ]:
weather.head()

Here, that columns is called station in our weather data. The next thing we should do is look at the shape to determine how to perform the join. 

In [ ]:
print(f"Weather: {weather.shape}")
print(f"Stations: {stations.shape}")

As a habbit and personal preference, I will typically take the take the table with less rows and join it to the table with more rows, but before we do that, it could also be beneficial to look at the number of unique values just to make sure that we don't lose any important information. 

In [ ]:
print(f"Unique stations from Weather: {weather.station.unique().shape}")
print(f"Unique stations from Stations: {stations.id.unique().shape}")

We could probably go a step further and double check that all of station IDs in `weather` are also present in `stations`, but I'm not that concerned about about that right now. We can move on to actually executing the join. There are several methods that you can use to perform the different joins, but I will showcase `merge` since it is capable of performing all the joins we are interested in for this class.

In [ ]:
joined_tables = weather.merge(stations, left_on='station', right_on='id')

In [ ]:
joined_tables.head()

Note that now we have all of the weather data from before (datatype, date, value, etc.) on the left side of the table, and all of the location data on the right (latitude, longitude, and elevation). Unfortunitely we do have two columns with identical informaiton, but this could be solved easiy by dropping one of the columns after the fact or by renaming the one column beforehand. If the joining columns have the same name, python only includes one column with that name. 

In [ ]:
weather.merge(stations.rename({'id':'station'}, axis='columns').drop('elevation', axis='columns'), on='station').head()

In [ ]:
stations.rename({'id':'station'}, axis='columns').drop('elevation', axis='columns')

Note that with this inner join, there are no missing values on either side of the table (at least none that isn't just bad or missing data)

In [ ]:
joined_tables.query('id.isna() or station.isna()').shape

We can probably for that to happen by performing a left or right join. 

In [ ]:
right_joined_tables = weather.merge(stations, left_on='station', right_on='id', how='right')
right_joined_tables.query("id.isna() or station.isna()")

Notice now that we have a set of rows for which the weather station data is present, but under the weather data, we have a bunch of missing values. This indicates to us that the some of the weather stations present in the `stations` dataframe are *not* present in the `weather` dataframe. Doing a quick bit of math, we can confirm that the missing rows matches up with what we expected. Performing an outer join is essentially the same as a left and a right join together, with the above behavior expected for both the left and right tables. 

## Window Functions

Window functions are a very interesting and useful function that can provide some insight. Sometimes, we want to know the maximum or the average of an entire column, but it can be just as interesting to know the rolling average of some sort of data. Window functions allow us to perform calculations  on a group of rows that are close to each otherin some way. Using the weather data, we can calculat the average rainfall over the past week. 

In [ ]:
snowfall = weather.query("datatype == 'PRCP' and station == 'GHCND:USW00094728'")\
    .set_index("date")\
    .assign(rolling_average=lambda x: x.value.rolling(7).mean())

In [ ]:
snowfall.query('rolling_average > 0')

## Aggregation
Another important function of data analysis is the idea of aggregation. This referes to the process of taking the data and rolling it up into a single value or set of values instead of looking at the individual measurements. We might want the summed, average, or maximum value of the dataset. We can use fairly simple function calls that we have acctually used to a degree before. I will use pivot tables to combine the different aggregations together in our weather data example. 

In [ ]:
weather.date = pd.to_datetime(weather.date)
weather_pivot = weather.set_index(['date', 'station'])\
    .pivot(columns='datatype', values='value')[['PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN']]
weather_pivot.query('station == "GHCND:USW00094728"')[['PRCP', 'SNOW']].sum()


The above gives us the totals over the entire dataset. We could make this perform the same logic to calculate things like the average per day, or find the days with the most or the least. The method calls are fairly intuitive, and can be quickly found in Panda's documentation. To make this slightly more intersting, we can use a method called `groupby` to group the data by some patitioning feature and perform the aggregation function on each group respectively. 

In [ ]:
weather_pivot.groupby("station")[['PRCP', 'SNOW']].sum()

or we could group by the date

In [ ]:
weather_pivot.groupby('date').min()

Additionally, you can group by multiple columns. It will create a set of nested groups that can be aggregated together for further separation of the metrics being calculated. 

There is also a pandas class called `Grouper` that seems to allow for even more complex grouping functionality 

# playground

In [ ]:
df = pd.DataFrame([range(10), [None for i in range(10)]]).T.rename({0: "index", 1: "empty"}, axis='columns')

In [ ]:
df.value_counts('index')

In [ ]:
df.isna()

In [ ]:
for col in df.isna().columns:
    if df[col].all(): 
        df = df.drop(col, axis='columns')

In [ ]:
df.filename = ['file.csv' for _ in range(10)]

In [ ]:
df

| | good | bad | okay | 
| - | - | - | - | 
file1.csv| 3 | 5 | 6 |
file2.csv| 3 | 5 | 6 |
file3.csv| 3 | 5 | 6 |

| cat | count |
| - | - |
| good| 3 | 
| bad | 5 | 
| okay | 34 | 

In [ ]:
csv_lines = [] 
with open('file.cs') as file: 
    line = file.readline()
    line = line.split(',')
    if len(line) == 5: 
        csv_lines.append(line)

df = pd.DataFrame(csv_lines[1:], columns=csv_lines[0])